# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

Импортируем необходимые для работы библиотеки:

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
df

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


Обычные признаки это количество МБ, минут, звонков и сообщений. Целевой - это тариф.

## Разбейте данные на выборки

Разобьем наш датасет на обучающую, валидационную и тестовую выборки:

In [4]:
df_train, df_valid = train_test_split(df, test_size=0.3, random_state=12345)

In [5]:
df_valid, df_test = train_test_split(df_valid,test_size = 0.5, random_state=12345)

Отделим целевые признаки от обычных:

In [6]:
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']
test_features = df_test.drop(['is_ultra'], axis=1)
test_target = df_test['is_ultra']
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

## Исследуйте модели

**Произведем проверку моделей для дерева решений:**

In [7]:
for depth in range (1,20):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    print("max_depth =", depth, ": ", end='')
    print(accuracy_score(target_valid, predictions_valid)) 

max_depth = 1 : 0.7510373443983402
max_depth = 2 : 0.7800829875518672
max_depth = 3 : 0.7863070539419087
max_depth = 4 : 0.7883817427385892
max_depth = 5 : 0.7614107883817427
max_depth = 6 : 0.7842323651452282
max_depth = 7 : 0.7821576763485477
max_depth = 8 : 0.7821576763485477
max_depth = 9 : 0.7780082987551867
max_depth = 10 : 0.7883817427385892
max_depth = 11 : 0.7904564315352697
max_depth = 12 : 0.7946058091286307
max_depth = 13 : 0.7821576763485477
max_depth = 14 : 0.7821576763485477
max_depth = 15 : 0.7593360995850622
max_depth = 16 : 0.7489626556016598
max_depth = 17 : 0.7510373443983402
max_depth = 18 : 0.7551867219917012
max_depth = 19 : 0.7551867219917012


Наилучший показательнь при глубине дерева, равной 12.

**Произведем проверку моделей Случайного Леса:**

In [8]:
best_model = None
best_result = 0
best_est = 0
for est in range(10, 101,10):
    model = RandomForestClassifier(random_state=12345, n_estimators=est) 
    model.fit(features_train,target_train) 
    result = model.score(features_valid, target_valid)
    if result > best_result:
        best_model = model.fit(features_train, target_train)
        best_result = result
        best_est = est
    print("result =", result, ": ", "est=", est )
print("Accuracy наилучшей модели на валидационной выборке:", best_result)
print("Количество деревьев:", best_est)

result = 0.7780082987551867 :  est= 10
result = 0.7904564315352697 :  est= 20
result = 0.7863070539419087 :  est= 30
result = 0.7946058091286307 :  est= 40
result = 0.7904564315352697 :  est= 50
result = 0.7925311203319502 :  est= 60
result = 0.7987551867219918 :  est= 70
result = 0.7966804979253111 :  est= 80
result = 0.7966804979253111 :  est= 90
result = 0.7966804979253111 :  est= 100
Accuracy наилучшей модели на валидационной выборке: 0.7987551867219918
Количество деревьев: 70


**Рассмотрим логистическую регрессию:**

In [9]:
model = LogisticRegression(random_state=12345) 
model.fit(features_train,target_train) 
result = model.score(features_valid,target_valid ) 
print("Accuracy модели логистической регрессии на валидационной выборке:", result)

Accuracy модели логистической регрессии на валидационной выборке: 0.6950207468879668


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


## Вывод:


**Исследовали три модели:** Дерево решений, Случайный Лес и логистическую регрессию.

**После исследования мы получили:**

 - У дерева решений при глубине 12 accuracy: 0.7946058091286307
 - У случайного леса при  70 деревьях accuracy: 0.7987551867219918
 - У логистической регрессии  accuracy 0.6950207468879668


Исходя из полученных результатов на данный момент, можно сказать, что **модель "Случайный лес" показывает лучший результат : при 70 деревьях, Accuracy: 0.7987551867219918**

## Проверьте модель на тестовой выборке

В качестве оптимальной модели возьмем RandomForestClassifier с количеством деревьев 70:

In [10]:
model = RandomForestClassifier(random_state=12345, n_estimators=70) 
model.fit(features_train, target_train)
test_predictions = model.predict(test_features)
score = accuracy_score(test_target, test_predictions)
print('Accuracy:', score)

Accuracy: 0.7908902691511387


При обучении модели на тестовой выборке, модель показала результат Accuracy: 0.7908902691511387.

## (бонус) Проверьте модели на адекватность

Для проверки модели на адекватность используем классификатор DummyClassifier, который делает прогнозы, используя простые правила. Он будет полезен в качестве простой основы для сравнения с другими (реальными) классификаторами. В качестве параметра используем «Most_frequent», который всегда предсказывает наиболее частую метку в обучающем наборе.

In [11]:
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(features_train, target_train)

DummyClassifier(constant=None, random_state=None, strategy='most_frequent')

In [12]:
dummy_clf.score(features_train, target_train)

0.6927523343708315

Мы видим, что **показатель нашей модели почти на 14% выше**, чем с данным классификатором, поэтому можно заключить, что наша модель вполне адекватна.